# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [3]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [4]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [165]:
aux = data.groupby(by = ["ProductName","CustomerID",]).sum()
aux.reset_index(inplace = True)

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [169]:
prd_cus_mtrx = pd.pivot_table(data = aux,values="Quantity", columns = "CustomerID" , index = ["ProductName"]).fillna(0)

## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [154]:
similarity = squareform(pdist(prd_cus_mtrx, metric = "euclidean"))
similarity = pd.DataFrame(1/(1+similarity), columns= prd_cus_mtrx.index, index= prd_cus_mtrx.index)

## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [173]:
top_5 = similarity.iloc[0].sort_values(ascending = False)[1:].head()
ids = []
for key, ele in top_5.items():
    print(set(data[data["CustomerID"] == key]["FirstName"]),ele)
    ids.append(key)
ids

{'Virgil'} 0.08704668331836253
{'Lana'} 0.08704668331836253
{'Chandra'} 0.08704668331836253
{'Kyle'} 0.08598344475655936
{'Bernard'} 0.08563786063744523


[264, 3535, 3317, 2503, 3305]

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [246]:
data_top5 = aux[aux["CustomerID"].isin(ids)]
data_top5.head()

,ProductName,CustomerID,SalesID,ProductID,Quantity
5,Anchovy Paste - 56 G Tube,3535,3904179,131,1
299,Appetizer - Mushroom Tart,2503,1275443,326,1
438,Appetizer - Sausage Rolls,2503,5130810,73,1
566,Apricots - Dried,3535,4143868,324,1
703,Apricots - Halves,264,3730301,278,1


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [247]:
top5_products = data_top5.groupby(by = "ProductName").sum().sort_values(by = "Quantity", ascending = False)
top5_products.reset_index(inplace=True)
top5_products.head()

,ProductName,CustomerID,SalesID,ProductID,Quantity
0,Butter - Unsalted,5820,13554503,1206,3
1,Wine - Ej Gallo Sierra Valley,6622,5892505,1326,3
2,Towels - Paper / Kraft,7104,5618699,957,3
3,Soup - Campbells Bean Medley,7116,9218997,1146,3
4,Wine - Blue Nun Qualitatswein,7104,9241577,807,3


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.